This notebook utilizes the Weather.gov API to generate the local forcast for a given latitude and longitude.


## Modules to Import



In [1]:
import sys
import urllib.request
import json
import datetime as dt

In [16]:
# If remounting (i.e. to reload updated data), run cell twice
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [17]:
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/')
from wtdlib import utils2 as utils

# Local Functions for Module

In [40]:
def calculate_utc_offset(utc_time, tz_string):
    """
    Local Function
    Input:
        utc_time: dt.datetime object with UTC timezone
        tz_string: string from weather.gov API
                'America/Los_Angeles', 'America/Denver',
                'America/Chicago', 'America/New_York'
    Output:
        offset: dt.timedelta object
    """
    if (tz_string == 'America/Los_Angeles' and utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-7)
        print('Pacific Daylight Time')
    elif (tz_string == 'America/Los_Angeles' and not utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-8)
        print('Pacific Standard Time')
    elif (tz_string == 'America/Denver' and utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-6)
        print('Mountian Daylight Time')
    elif (tz_string == 'America/Denver' and not utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-7)
        print('Mountain Standard Time')
    elif (tz_string == 'America/Chicago' and utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-5)
        print('Central Daylight Time')
    elif (tz_string == 'America/Chicago' and not utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-6)
        print('Central Standard Time')
    elif (tz_string == 'America/New_York' and utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-4)
        print('Eastern Daylight Time')
    elif (tz_string == 'America/New_York' and not utils.isdst(utc_time)):
        offset = dt.timedelta(hours=-5)
        print('Eastern Standard Time')
    else:
        offset = dt.timedelta(hours=0)
        print('No Offset, retain UTC')
    return offset

# Weather.gov Query

In [49]:
# Latitude & Longitude of San Francisco, CA: 37.7749° N, 122.4194° W
# home = [37.7749, -122.4194]

# Latitude & Longitude of Denver, CO: 39.7392° N, 104.9903° W
home = [39.7392, -104.9903]

# Latitude & Longitude of Chicago, IL: 41.8781° N, 87.6298° W
# home = [41.8781, -87.6298]

# Latitude & Longitude of Boston, MA: 42.3555° N, 71.0565° W
# home = [42.3555, -71.0565]

# API Directions to access Weather Data
# https://www.weather.gov/documentation/services-web-api
url_points = f"https://api.weather.gov/points/{home[0]},{home[1]}"
print(url_points)

with urllib.request.urlopen(url_points) as response:
   text_points = response.read()
json_points = json.loads(text_points)

# Obtain Forecast URL from json_points
url_forecast = json_points['properties']['forecast']
print(url_forecast)

with urllib.request.urlopen(url_forecast) as response:
   text_forecast = response.read()
json_forecast = json.loads(text_forecast)

# Obtain Weather Station Location from json_points
rel_loc = json_points['properties']['relativeLocation']
city = rel_loc['properties']['city']
state = rel_loc['properties']['state']
coords = rel_loc['geometry']['coordinates']


# Obtain Collection Time from json_forecast
time_utc = dt.datetime.strptime(
    json_forecast['properties']['updateTime'],
    '%Y-%m-%dT%H:%M:%S%z'
)

time_utc = dt.datetime.strptime(
    json_forecast['properties']['updateTime'],
    '%Y-%m-%dT%H:%M:%S%z')
tz = json_points['properties']['timeZone']
tz_offset = calculate_utc_offset(time_utc, tz)
time_local = time_utc + tz_offset
# time_local = utils.convert_utc_to_local(time_utc, tz)

https://api.weather.gov/points/39.7392,-104.9903
https://api.weather.gov/gridpoints/BOU/63,62/forecast
Mountian Daylight Time


In [48]:
print('Input Coordinates')
print(f'Latitude: {home[0]} \t Longitude: {home[1]}')
print()
print('Station Coordinates')
print(f'City: {city} \t State: {state}')
print(f'Latitude: {coords[1]} \t Longitude: {coords[0]}')
print()
print('Data Generation Time')
print('UTC: ', time_utc.strftime('%A, %B %d, %Y; %I:%M %p'))#; %z')
print(tz + ': ', time_local.strftime('%A, %B %d, %Y; %I:%M %p'))

Input Coordinates
Latitude: 39.7392 	 Longitude: -104.9903

Station Coordinates
City: Glendale 	 State: CO
Latitude: 39.703052 	 Longitude: -104.936157

Data Generation Time
UTC:  Monday, April 28, 2025; 08:16 PM
America/Denver:  Monday, April 28, 2025; 02:16 PM


In [33]:
# Print Forecast
for i in range(len(json_forecast['properties']['periods'])):
    print(json_forecast['properties']['periods'][i]['name'])
    print(json_forecast['properties']['periods'][i]['detailedForecast'])
    print()

Tonight
Mostly clear, with a low around 53. South wind around 6 mph.

Tuesday
Sunny, with a high near 75. Southwest wind 6 to 20 mph.

Tuesday Night
Mostly cloudy, with a low around 60. Southwest wind 15 to 20 mph.

Wednesday
Mostly sunny, with a high near 67. West wind 12 to 15 mph.

Wednesday Night
Partly cloudy, with a low around 46. North wind 6 to 10 mph.

Thursday
Mostly sunny, with a high near 61. East wind 5 to 8 mph.

Thursday Night
A chance of rain after 1am. Mostly cloudy, with a low around 50. South wind around 9 mph. Chance of precipitation is 30%.

Friday
A chance of rain. Mostly cloudy, with a high near 72. South wind 9 to 15 mph. Chance of precipitation is 30%.

Friday Night
A chance of rain. Mostly cloudy, with a low around 56. Southwest wind 9 to 15 mph. Chance of precipitation is 40%.

Saturday
A chance of rain. Partly sunny, with a high near 67. Southwest wind around 12 mph. Chance of precipitation is 50%.

Saturday Night
A chance of rain before 9pm, then a slight c